# Scraping images from mapbox satelitar view  
Mapbox Free tier allows 50,000 requests per month : consider logging how many images have been scraped already to not exceed this limit.  


Pipeline : 
- Define area of interest (bounding box) from coordinates (lat, lon)
- Get the satelitar view of the bounding box from mapbox API
- Crop the images to remove watermark
- Save the images to disk
- Get the bounding boxes of buildings from OSM for the area of interest
- Save the bounding boxes to disk
- Clean overlapping bounding boxes (remove smaller ones or merge them)
- Save the cleaned bounding boxes to disk

The areas of interest have been picked manually :
- Top 20 most populous cities in France
- 100 rural areas coordinates picked randomly across France
- Maybe some random forest to get images without buildings (negative samples)

For the cities : 
- We define a bounding box around the city center with a fixed size (e.g., 10km x 10km)
- We then take random points from this bounding box to scrape images

Then, we can use the pipeline with these points to scrape images and get bounding boxes of buildings.

importing packages

In [20]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

CONFIG

In [ ]:
# Load environment variables from .env file
load_dotenv()

# MAPBOX
MAPBOX_ACCESS_TOKEN = os.getenv("MAPBOX_ACCESS_TOKEN")

# Image parameters
WIDTH = 512
HEIGHT = 512 + 30  # extra space to then delete bottom part with watermark
ZOOM = 18  # 18-19 to see buildings, swimming pools clearly

PATH = Path().cwd().parent
BASE_DIR = PATH / "data"
RAW_DIR = BASE_DIR / "raw_images"
CROPPED_DIR = BASE_DIR / "cropped_images"

RAW_POLYGON_DIR = BASE_DIR / "raw_polygons"
CLEANED_POLYGON_DIR = BASE_DIR / "cleaned_polygons"

sys.path.append(str(PATH))

First, we get all the coordinates of the areas of interest from the two csv files.

In [22]:
# to import csv files
import pandas as pd

Rural areas coordinates file : "rural_coords.csv"

In [23]:
FILE_NAME: str = "rural_coords.csv"
rural_coords: pd.DataFrame = pd.read_csv(
    PATH / FILE_NAME,
    sep = ",",
    header=None
)
rural_coords.columns = ["latitude", "longitude"]

In [24]:
# checking the rural coordinates file
print(f"Number of rural areas: {len(rural_coords)}")
rural_coords.head()

Number of rural areas: 100


,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


Cities coordinates file : "city_coords.csv"

In [25]:
FILE_NAME: str = "city_coords.csv"
city_coords: pd.DataFrame = pd.read_csv(
    PATH / FILE_NAME,
    sep = ","
)

In [26]:
# checking the city coordinates file
print(f"Number of cities: {len(city_coords)}")
city_coords.head()

Number of cities: 19


,city,lat,lng
0,Paris,48.8567,2.3522
1,Bordeaux,44.8400,-0.5800
2,Marseille,43.2964,5.3700
3,Lyon,45.7600,4.8400
4,Toulouse,43.6045,1.4440


For the cities, we now create a bounding box around the city center with a fixed size (e.g., 10km x 10km) and then take random points from this bounding box to scrape images.

In [27]:
from miscellaneous import create_bbox_from_city_coordinates

city_coords_bbox: dict[str, dict[str, float]] = {}
# {city :  {"west": .., "south": .., "east": .., "north": ..}

for _, row in city_coords.iterrows():
    city_name: str = row["city"]
    latitude: float = row["lat"]
    longitude: float = row["lng"]
    bbox: dict[str, float] = create_bbox_from_city_coordinates(
        latitude,
        longitude,
        width=10,
        height=10
    )
    # width and height are in km
    city_coords_bbox[city_name] = bbox


In [28]:
city_coords_df = pd.DataFrame.from_dict(city_coords_bbox, orient="index")
city_coords_df.head()

,west,south,east,north
Paris,2.284676,48.811347,2.419842,48.902013
Bordeaux,-0.640288,44.792973,-0.519611,44.886994
Marseille,5.306649,43.252774,5.433447,43.339990
Lyon,4.774246,45.716018,4.905859,45.803944
Toulouse,1.383371,43.558618,1.504726,43.650350


Now that we have the bounding boxes for each city, we can get random points from these bounding boxes to scrape images.

In [29]:
from miscellaneous import get_random_points_in_bbox

city_random_points: list[dict[str, float]] = []
for city, bbox in city_coords_bbox.items():
    random_points: list[dict[str, float]] = get_random_points_in_bbox(
        bbox,
        number_of_points=10
    )
    # 10 coordinates/images for each city -> 20 * 10 = 200 images
    city_random_points.extend(random_points)
city_random_points_df = pd.DataFrame(city_random_points)
city_random_points_df.columns = ["latitude", "longitude"]

Now that we have all the coordinates (rural + cities), we combine them into a single list of coordinates to use in the pipeline.

In [30]:
# we combine both rural and city coordinates into a single dataframe
all_coords_df = pd.DataFrame(columns=["latitude", "longitude"])
all_coords_df = pd.concat(
    [
        rural_coords,
        city_random_points_df
    ],
    ignore_index=True
)

In [31]:
all_coords_df.head()

,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


We can now use the pipeline to scrape images and get bounding boxes of buildings.

In [34]:
from miscellaneous import create_bbox_from_coordinates

for _, row in all_coords_df.sample(5).iterrows():
    print(f"Latitude: {row['latitude']}, Longitude: {row['longitude']}")
    # now we get bbox from coordinates
    bbox: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude']
    )
    # other parameters include :
    # - img_height: int = 512 + 30,
    # - img_width: int = 512,
    # - pixel_size: float = 0.4
    # returns : {"west": .., "south": .., "east": .., "north": ..}
    print(f"BBox: {bbox}\n")


Latitude: 47.287870961316735, Longitude: 5.038744385244625
BBox: {'west': 5.0373530074758355, 'south': 47.28691921196537, 'east': 5.040135812308603, 'north': 47.2888226936139}

Latitude: 48.84343574755057, Longitude: 2.3769586279451707
BBox: {'west': 2.37557510922138, 'south': 48.842453627806805, 'east': 2.3783421988116014, 'north': 48.84441785042915}

Latitude: 48.36348109495955, Longitude: -1.327774380474659
BBox: {'west': -1.32907461999394, 'south': 48.36245682436928, 'east': -1.3264740903343328, 'north': 48.36450535050039}

Latitude: 48.77022737185834, Longitude: 2.1604914828381756
BBox: {'west': 2.1591139831862916, 'south': 48.76924268867019, 'east': 2.1618690344649862, 'north': 48.77121203831489}

Latitude: 47.33511134022042, Longitude: 5.059515070485326
BBox: {'west': 5.058122100057363, 'south': 47.33415985869076, 'east': 5.06090809029541, 'north': 47.33606280465974}



In [50]:
# testing on 5 random coordinates from the all_coords_df dataframe
import osmnx as ox

from miscellaneous.ask_mapbox_for_image import (
    ask_mapbox_for_image
)
from miscellaneous.clean_overlapping_bboxes import clean_overlapping_bboxes

five_random_points: pd.DataFrame = all_coords_df.sample(5)
image_width_height: dict[str, int] = {"width": WIDTH, "height": HEIGHT}
for i, row in five_random_points.iterrows():
    print(f"Latitude: {row['latitude']}, Longitude: {row['longitude']}")
    # now we get bbox from coordinates
    bbox_for_mapbox: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude']
    )
    # other parameters include :
    # - img_height: int = 512 + 30, # extra space to then delete bottom part with watermark
    # - img_width: int = 512,
    # - pixel_size: float = 0.4
    # returns : {"west": .., "south": .., "east": .., "north": ..}
    print(f"BBox: {bbox_for_mapbox}\n")

    # # now we can use this bbox to download the image from mapbox static api
    # raw_image_file: Path = RAW_DIR / f"image_{i}.png"
    # image = ask_mapbox_for_image(
    #     output_file=raw_image_file,
    #     image_width_height=image_width_height,
    #     mapbox_token=MAPBOX_ACCESS_TOKEN,
    #     bounding_box=bbox_for_mapbox,
    # )
    # # parameters:
    # # - output_file: Path,
    # # - image_width_height: dict[str, int],
    # # - mapbox_token: str,
    # # - bounding_box: dict[str, float],
    # # - request_timeout: int = 10
    # print(f"Image saved to: {raw_image_file}\n")    # displays the image
    # display(image)

    # # cutting the watermark part : cutting 30 pixel from the bottom
    # cropped_image = image.crop((0, 0, WIDTH, HEIGHT - 30))

    # # saving the image to the cropped directory
    # cropped_image_file: Path = CROPPED_DIR / f"image_{i}.png"
    # image.save(cropped_image_file)


    # POLYGONS :
    # First, we get the polygons from OpenStreetMap.
    bbox_for_polygons: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude'],
        img_height=512,
        img_width=512 # we removed the 30 pixels from the bottom for the watermark
    )
    bbox_for_polygons = tuple(bbox_for_polygons.values())
    gdf = ox.features_from_bbox(bbox=bbox_for_polygons, tags={"building": True})

    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    gdf = gdf.reset_index(drop=True)

    # we save the buildings polygons from OpenStreetMap
    raw_polygon_file = RAW_POLYGON_DIR / f"buildings_{i}.geojson"
    gdf.to_file(raw_polygon_file, driver="GeoJSON")

    # we clean the polygons using "clean_overlapping_bboxes"
    clean_polygon_file = CLEANED_POLYGON_DIR / f"buildings_{i}.geojson"
    gdf = clean_overlapping_bboxes(
        gdf = gdf,
        what_to_do_on_overlap = "merge"
    )
    gdf.to_file(clean_polygon_file, driver="GeoJSON")


Latitude: 50.69104556253334, Longitude: 3.171766854088163
BBox: {'west': 3.170316979031872, 'south': 50.69007482672573, 'east': 3.1732167852653137, 'north': 50.69201628002082}

Latitude: 50.686801259216836, Longitude: 3.1783063026513556
BBox: {'west': 3.176856423723949, 'south': 50.68583059353094, 'east': 3.179756237690611, 'north': 50.687771906582505}

Latitude: 49.73600265026652, Longitude: 2.894285669716014
BBox: {'west': 2.8928681458790693, 'south': 49.73502743880539, 'east': 2.8957032475762117, 'north': 49.736977844109376}

Latitude: 47.90648498803779, Longitude: 1.9206921719612369
BBox: {'west': 1.9193415452498805, 'south': 47.90549691556853, 'east': 1.9220428489447892, 'north': 47.90747304438572}

Latitude: 50.718917465396544, Longitude: 3.2052276352564384
BBox: {'west': 3.203776306278471, 'south': 50.717947160496614, 'east': 3.206679020420174, 'north': 50.7198877519451}



InsufficientResponseError: No matching features. Check query location, tags, and log.